#### Libraries

In [1]:
import numpy as np

#### Initialization

In [2]:
A = np.array([[5., 1., -1.], [10., 3., -5.], [2., 8., 6.]])
b = np.array([1.,2.,3.])
A

array([[  5.,   1.,  -1.],
       [ 10.,   3.,  -5.],
       [  2.,   8.,   6.]])

### Compute LU decomposition

In [3]:
def LU(A):
    N = A.shape[0]
    Upper = A.copy()
    Lower = np.identity(N)
    for i in range(N-1):
        for k in range(i+1,N):
            Lower[k,i] = Upper[k,i]/Upper[i,i]
            for j in range(i,N):
                Upper[k,j] = Upper[k,j] - Lower[k,i]*Upper[i,j]
    return Lower, Upper

In [4]:
L,U = LU(A)

#### Test

In [5]:
L

array([[ 1. ,  0. ,  0. ],
       [ 2. ,  1. ,  0. ],
       [ 0.4,  7.6,  1. ]])

In [6]:
U

array([[  5. ,   1. ,  -1. ],
       [  0. ,   1. ,  -3. ],
       [  0. ,   0. ,  29.2]])

In [7]:
np.dot(L,U)-A

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

### Back and Forward Substitution

In [8]:
def forwardsub(L,b):
    N = L.shape[0]
    y = np.zeros(N)
    for i in range(N):
        s = 0
        for j in range(i):
                s += L[i,j]*y[j]
        y[i] = (1/L[i,i])*(b[i] - s)
    return y

In [9]:
def backsub(U,y):
    N = U.shape[0]
    x = np.zeros(N)
    for i in range(N)[::-1]:
        s = 0
        for j in range(i+1,N):
                s += U[i,j]*x[j]
        x[i] = (1/U[i,i])*(y[i] - s)
    return x

#### Test

In [10]:
y = forwardsub(L,b)
y

array([ 1. ,  0. ,  2.6])

In [11]:
x = backsub(U,y)
x

array([ 0.16438356,  0.26712329,  0.0890411 ])

In [12]:
np.dot(A,x)

array([ 1.,  2.,  3.])

### All together

In [13]:
def solveLU(L,U,b):
    N = L.shape[0]
    x = np.zeros(N)
    y = np.zeros(N)
    for i in range(N):
        s = 0
        for j in range(i):
            s += L[i,j]*y[j]
        y[i] = (1/L[i,i])*(b[i] - s)

    for i in range(N)[::-1]:
        s = 0
        for j in range(i+1,N):
            s += U[i,j]*x[j]
        x[i] = (1/U[i,i])*(y[i] - s)
    return x

### Test

In [14]:
a = np.array([np.random.rand(1)[0]*10,np.random.rand(1)[0]*10,np.random.rand(1)[0]*10])
b = np.array([np.random.rand(1)[0]*10,np.random.rand(1)[0]*10,np.random.rand(1)[0]*10])
c = np.array([np.random.rand(1)[0]*10,np.random.rand(1)[0]*10,np.random.rand(1)[0]*10])
d = np.array([np.random.rand(1)[0]*10,np.random.rand(1)[0]*10,np.random.rand(1)[0]*10])
e = np.array([np.random.rand(1)[0]*10,np.random.rand(1)[0]*10,np.random.rand(1)[0]*10])
a

array([ 8.56182935,  0.32697142,  0.59319618])

In [15]:
[np.dot(A,solveLU(L,U,a)) - a,
np.dot(A,solveLU(L,U,b)) - b,
np.dot(A,solveLU(L,U,c)) - c,
np.dot(A,solveLU(L,U,d)) - d,
np.dot(A,solveLU(L,U,e)) - e]

[array([  0.00000000e+00,   6.66133815e-16,   2.39808173e-14]),
 array([  1.77635684e-15,   1.55431223e-15,   9.76996262e-15]),
 array([  0.00000000e+00,   0.00000000e+00,  -1.24344979e-14]),
 array([  0.00000000e+00,   8.88178420e-16,  -1.77635684e-15]),
 array([  1.77635684e-15,   3.55271368e-15,   2.22044605e-15])]

### Now with scaled partial pivoting

In [16]:
A = np.array([[5., 1., -1.], [10., 2., -1.], [2., 8., 6.]])
b = np.array([1.,2.,3.])
A

array([[  5.,   1.,  -1.],
       [ 10.,   2.,  -1.],
       [  2.,   8.,   6.]])

In [17]:
def pivoting(i,A,N):
    d = np.zeros(N)
    for j in range(i,N):
        d[j] = np.abs(A[j,i])
        for k in range(i+1,N):
            if d[j] < np.abs(A[j,k]):
                d[j] = np.abs(A[j,k])
    pivotrow = i
    pivot = np.abs(A[i,i])/d[i]
    for j in range(i+1,N):
        if pivot < abs(A[j,i])/d[j]:
            pivot = abs(A[j,i])/d[j]
            pivotrow = j
    return pivotrow   

In [18]:
def projector(p):
    N = len(p)
    P1 = np.identity(N)
    P2 = np.identity(N)
    for i in range(N):
        P1[:][i] = P2[:][p[i]]
    return P1

In [19]:
def PLU(A):
    N = A.shape[0]                # Preliminary Definitions
    Upper = A.copy()
    Lower = np.zeros(A.shape)
    p = range(N)
    
    for i in range(N-1):          # Perform the pivoting
        l = pivoting(i,Upper,N)
        if l != i:
            tempVecUP = np.zeros(N)
            tempVecDOWN = np.zeros(N)
            tempVal = p[i]
            p[i] = l
            p[l] = tempVal
            for n in range(N):
                tempVecUP[n] = Upper[i,n]
                tempVecDOWN[n] = Lower[i,n]
                Upper[i,n] = Upper[l,n]
                Lower[i,n] = Lower[l,n]                
                Upper[l,n] = tempVecUP[n]
                Lower[l,n] = tempVecDOWN[n]                
        
        for k in range(i+1,N):        # Gaussian Elim
            Lower[k,i] = Upper[k,i]/Upper[i,i]
            for j in range(i,N):
                Upper[k,j] = Upper[k,j] - Lower[k,i]*Upper[i,j]
    Lower += np.identity(N)
    P = projector(p)
    return p, P, Lower, Upper 

In [20]:
p,P,L,U = PLU(A)

#### Test

In [21]:
p

[0, 2, 1]

In [22]:
P

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.]])

In [23]:
L

array([[ 1. ,  0. ,  0. ],
       [ 0.4,  1. ,  0. ],
       [ 2. ,  0. ,  1. ]])

In [24]:
U

array([[ 5. ,  1. , -1. ],
       [ 0. ,  7.6,  6.4],
       [ 0. ,  0. ,  1. ]])

In [27]:
np.dot(L,U)

array([[  5.,   1.,  -1.],
       [  2.,   8.,   6.],
       [ 10.,   2.,  -1.]])

In [28]:
A

array([[  5.,   1.,  -1.],
       [ 10.,   2.,  -1.],
       [  2.,   8.,   6.]])

#### All together

In [29]:
def solvePLU(P,L,U,b):
    N = L.shape[0]
    x = np.zeros(N)
    y = np.zeros(N)
    b = np.dot(P,b)
    for i in range(N):
        s = 0
        for j in range(i):
            s += L[i,j]*y[j]
        y[i] = (1/L[i,i])*(b[i] - s)

    for i in range(N)[::-1]:
        s = 0
        for j in range(i+1,N):
            s += U[i,j]*x[j]
        x[i] = (1/U[i,i])*(y[i] - s)
    return x

In [ ]:
[np.dot(A,solvePLU(P,L,U,a)) - a,
np.dot(A,solvePLU(P,L,U,b)) - b,
np.dot(A,solvePLU(P,L,U,c)) - c,
np.dot(A,solvePLU(P,L,U,d)) - d,
np.dot(A,solvePLU(P,L,U,e)) - e]